In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
import calendar
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
#Define presto credentials
presto_host = 'bi-presto.serving.data.production.internal'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [3]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, customer_location_latitude, 
pickup_location_hex_8, pickup_location_latitude, pickup_location_longitude, drop_location_hex_8, drop_location_latitude, drop_location_longitude,
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason
from orders.order_logs_snapshot
where yyyymmdd >= '20230306'
and yyyymmdd <= '20230310'
and city_name in ('Jaipur')
and service_obj_service_name = 'Link'
and hour in ('08', '09', '10', '11', '17', '18', '19', '20', '21')
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [4]:
#Create a copy of the df retrived from presto
df = df_copy.copy()
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason
0,640962b8b122592c61e0faff,3.577,883da218e5fffff,26.895327,75.778862,Jaipur,883da218e5fffff,26.895376,883da218e5fffff,26.895322,75.778735,883da219ddfffff,26.881447,75.767066,75.778870,100816,10,1000,Link,10:00,4,20230309,customerCancelled,Taking longer than expected
1,6409657876454e7fc02a2913,1.098,883da20b15fffff,26.821011,75.836044,Jaipur,883da219b9fffff,26.858522,883da219b9fffff,26.858523,75.793449,883da20b15fffff,26.821011,75.836044,75.793449,102000,10,1015,Link,10:00,4,20230309,dropped,None
2,640966438129a356a3de14ec,NaN,883da218c7fffff,26.920370,75.788712,Jaipur,883da218c7fffff,26.920340,883da218c7fffff,26.920198,75.788902,883da21ab7fffff,26.939203,75.788406,75.788719,102323,10,1015,Link,10:00,4,20230309,customerCancelled,order cancelled before rider accepted
3,640966aa3d612b1ceefd961f,1.706,883da218ebfffff,26.915789,75.785126,Jaipur,883da2199bfffff,26.876308,883da2199bfffff,26.876314,75.800406,883da218ebfffff,26.915789,75.785126,75.800407,102506,10,1015,Link,10:00,4,20230309,dropped,None
4,6409674e5503d46c537f2b79,NaN,883da21991fffff,26.871908,75.798004,Jaipur,883da21991fffff,26.871910,883da21991fffff,26.872082,75.798485,883da2198bfffff,26.872141,75.782187,75.798004,102750,10,1015,Link,10:00,4,20230309,customerCancelled,order cancelled before rider accepted


In [5]:
#Make columns for second and minute and date
df['second'] = df['hhmmss'].apply(lambda x: x[4:])
df['minute'] = df['hhmmss'].apply(lambda x: x[2:4])
df['date'] = df['yyyymmdd'].apply(lambda x: x[6:])
df['count'] = 1
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count
0,640962b8b122592c61e0faff,3.577,883da218e5fffff,26.895327,75.778862,Jaipur,883da218e5fffff,26.895376,883da218e5fffff,26.895322,75.778735,883da219ddfffff,26.881447,75.767066,75.778870,100816,10,1000,Link,10:00,4,20230309,customerCancelled,Taking longer than expected,16,08,09,1
1,6409657876454e7fc02a2913,1.098,883da20b15fffff,26.821011,75.836044,Jaipur,883da219b9fffff,26.858522,883da219b9fffff,26.858523,75.793449,883da20b15fffff,26.821011,75.836044,75.793449,102000,10,1015,Link,10:00,4,20230309,dropped,None,00,20,09,1
2,640966438129a356a3de14ec,NaN,883da218c7fffff,26.920370,75.788712,Jaipur,883da218c7fffff,26.920340,883da218c7fffff,26.920198,75.788902,883da21ab7fffff,26.939203,75.788406,75.788719,102323,10,1015,Link,10:00,4,20230309,customerCancelled,order cancelled before rider accepted,23,23,09,1
3,640966aa3d612b1ceefd961f,1.706,883da218ebfffff,26.915789,75.785126,Jaipur,883da2199bfffff,26.876308,883da2199bfffff,26.876314,75.800406,883da218ebfffff,26.915789,75.785126,75.800407,102506,10,1015,Link,10:00,4,20230309,dropped,None,06,25,09,1
4,6409674e5503d46c537f2b79,NaN,883da21991fffff,26.871908,75.798004,Jaipur,883da21991fffff,26.871910,883da21991fffff,26.872082,75.798485,883da2198bfffff,26.872141,75.782187,75.798004,102750,10,1015,Link,10:00,4,20230309,customerCancelled,order cancelled before rider accepted,50,27,09,1


In [6]:
#Load data from cluster table
q = """
select * from datasets.city_cluster_hex
where city in ('Jaipur')
and resolution = 8
"""
#Load data into pandas table
df_cluster_copy = pd.read_sql(q, connection)

In [7]:
cluster_mapping = df_cluster_copy.copy()
cluster_mapping = cluster_mapping.rename(columns = {'hex_id':'pickup_location_hex_8'})
df = df.merge(cluster_mapping[['pickup_location_hex_8', 'cluster']], on = 'pickup_location_hex_8', how = 'left')
cluster_mapping = cluster_mapping.rename(columns = {'pickup_location_hex_8':'drop_location_hex_8'})
df = df.rename(columns = {'cluster':'pickup_cluster'})
df = df.merge(cluster_mapping[['drop_location_hex_8', 'cluster']], on = 'drop_location_hex_8', how = 'left')
df = df.rename(columns = {'cluster':'drop_cluster'})
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count,pickup_cluster,drop_cluster
0,640962b8b122592c61e0faff,3.577,883da218e5fffff,26.895327,75.778862,Jaipur,883da218e5fffff,26.895376,883da218e5fffff,26.895322,75.778735,883da219ddfffff,26.881447,75.767066,75.778870,100816,10,1000,Link,10:00,4,20230309,customerCancelled,Taking longer than expected,16,08,09,1,Swej Farm_JAI,Swej Farm_JAI
1,6409657876454e7fc02a2913,1.098,883da20b15fffff,26.821011,75.836044,Jaipur,883da219b9fffff,26.858522,883da219b9fffff,26.858523,75.793449,883da20b15fffff,26.821011,75.836044,75.793449,102000,10,1015,Link,10:00,4,20230309,dropped,None,00,20,09,1,Durgapura_JAI,Sheopur_JAI
2,640966438129a356a3de14ec,NaN,883da218c7fffff,26.920370,75.788712,Jaipur,883da218c7fffff,26.920340,883da218c7fffff,26.920198,75.788902,883da21ab7fffff,26.939203,75.788406,75.788719,102323,10,1015,Link,10:00,4,20230309,customerCancelled,order cancelled before rider accepted,23,23,09,1,Railway_Station_JAI,Vidhyadhar Nagar_JAI
3,640966aa3d612b1ceefd961f,1.706,883da218ebfffff,26.915789,75.785126,Jaipur,883da2199bfffff,26.876308,883da2199bfffff,26.876314,75.800406,883da218ebfffff,26.915789,75.785126,75.800407,102506,10,1015,Link,10:00,4,20230309,dropped,None,06,25,09,1,Jhalana Dungari_JAI,Sodala_JAI
4,6409674e5503d46c537f2b79,NaN,883da21991fffff,26.871908,75.798004,Jaipur,883da21991fffff,26.871910,883da21991fffff,26.872082,75.798485,883da2198bfffff,26.872141,75.782187,75.798004,102750,10,1015,Link,10:00,4,20230309,customerCancelled,order cancelled before rider accepted,50,27,09,1,Gopalpura_JAI,Triveni_Nagar_JAI


In [8]:
#Filter the df on morning and evening peak hours
df_morning = df[df['hour'].isin(['08', '09', '10', '11'])]
df_evening = df[df['hour'].isin(['17', '18', '19', '20', '21'])]

#Groupby on hex and quarter hour to get the number of orders coming in every hex
df_morning_grouped = df_morning.groupby(['yyyymmdd', 'hour', 'pickup_location_hex_8'])['order_id'].nunique().reset_index()
df_evening_grouped = df_evening.groupby(['yyyymmdd', 'hour', 'pickup_location_hex_8'])['order_id'].nunique().reset_index()
df_morning_grouped = df_morning_grouped.rename(columns = {'order_id':'num_of_orders'})
df_evening_grouped = df_evening_grouped.rename(columns = {'order_id':'num_of_orders'})
df_evening_grouped.head()

,yyyymmdd,hour,pickup_location_hex_8,num_of_orders
0,20230306,17,883da2014dfffff,3
1,20230306,17,883da20165fffff,4
2,20230306,17,883da20169fffff,3
3,20230306,17,883da2016bfffff,1
4,20230306,17,883da20225fffff,1


In [9]:
cluster_grouped_morning = df[df['hour'].isin(['08', '09', '10', '11'])].groupby(['pickup_cluster'])['order_id'].nunique().reset_index()
cluster_grouped_evening = df[df['hour'].isin(['17', '18', '19', '20', '21'])].groupby(['pickup_cluster'])['order_id'].nunique().reset_index()
cluster_grouped_morning = cluster_grouped_morning.sort_values(by = 'order_id', ascending=False)
cluster_grouped_evening = cluster_grouped_evening.sort_values(by = 'order_id', ascending=False)

top_clusters_morning = cluster_grouped_morning.head(40)
top_clusters_morning['orders_per_day'] = top_clusters_morning['order_id'].apply(lambda x: x/5)
top_clusters_morning['orders_per_hour'] = top_clusters_morning['orders_per_day'].apply(lambda x: x/4)
top_clusters_morning['orders_per_20_sec_batch'] = top_clusters_morning['orders_per_hour'].apply(lambda x: x/180)

top_clusters_evening = cluster_grouped_evening.head(40)
top_clusters_evening['orders_per_day'] = top_clusters_evening['order_id'].apply(lambda x: x/5)
top_clusters_evening['orders_per_hour'] = top_clusters_evening['orders_per_day'].apply(lambda x: x/4)
top_clusters_evening['orders_per_20_sec_batch'] = top_clusters_evening['orders_per_hour'].apply(lambda x: x/180)
top_clusters_evening

,pickup_cluster,order_id,orders_per_day,orders_per_hour,orders_per_20_sec_batch
21,Gopalpura_JAI,3276,655.2,163.80,0.910000
43,Lalkothi_JAI,2982,596.4,149.10,0.828333
58,Nirman_Nagar_JAI,1893,378.6,94.65,0.525833
49,Malviya_Nagar_JAI 2,1710,342.0,85.50,0.475000
64,Railway_Station_JAI,1324,264.8,66.20,0.367778
51,Mansarovar 1_JAI,1205,241.0,60.25,0.334722
91,Vidhyadhar Nagar_JAI,1142,228.4,57.10,0.317222
88,Vaishali Nagar_JAI 2,1140,228.0,57.00,0.316667
0,,1101,220.2,55.05,0.305833
73,Sindhi_Camp_JAI,1065,213.0,53.25,0.295833


In [10]:
top_clusters_morning

,pickup_cluster,order_id,orders_per_day,orders_per_hour,orders_per_20_sec_batch
21,Gopalpura_JAI,2231,446.2,111.55,0.619722
43,Lalkothi_JAI,1452,290.4,72.60,0.403333
58,Nirman_Nagar_JAI,1166,233.2,58.30,0.323889
49,Malviya_Nagar_JAI 2,1162,232.4,58.10,0.322778
91,Vidhyadhar Nagar_JAI,1039,207.8,51.95,0.288611
55,Narayan Vihar_JAI,962,192.4,48.10,0.267222
64,Railway_Station_JAI,915,183.0,45.75,0.254167
29,Jagatpura_JAI,885,177.0,44.25,0.245833
0,,802,160.4,40.10,0.222778
51,Mansarovar 1_JAI,726,145.2,36.30,0.201667


In [11]:
df_morning = df_morning.merge(df_morning_grouped, on = ['yyyymmdd', 'hour', 'pickup_location_hex_8'], how = 'left')
df_evening = df_evening.merge(df_evening_grouped, on = ['yyyymmdd', 'hour', 'pickup_location_hex_8'], how = 'left')

In [12]:
df_morning['weekday_name'] = df_morning['weekday'].apply(lambda x: calendar.day_name[(x-1)])
df_evening['weekday_name'] = df_evening['weekday'].apply(lambda x: calendar.day_name[(x-1)])

In [13]:
#Query data for mismatch

q_online_captains = """
select * from pricing.cluster_temporal_hotness
where service_detail_id = '5ba1f6d0be5b826084cc7202'
"""


#Load data into pandas table
df_mismatch_copy = pd.read_sql(q_online_captains, connection)

In [14]:
#View the dataset
df_mismatch = df_mismatch_copy.copy()
df_mismatch.head()

,hotness_category,pickupclusters,week_period,time_period,temporal_id,total_footfall,total_revenue,hex_count,total_revenue_normalized,normalized_revenue_contribution_percentage,cum_revenue_contribution_percentage,revenue_contribution_percentile_rank,temporal_stress,temporal_mismatch,spatial_stress,spatial_mismatch,cluster_hotness_score,run_date,strategy_version,service_detail_id
0,hot,Railway Station_JAI,Monday,1400,Monday-1400,238,16719.0,1,16719.000000,23.005786,23.005786,0.000000,0.0,0.0,0.0,0.0,0.0,20221025,HOT_COLD_2_0,5ba1f6d0be5b826084cc7202
1,hot,Gurjar Ki Thadi_JAI,Monday,1400,Monday-1400,182,8980.0,2,4490.000000,6.178359,29.184145,1.886792,0.0,0.0,0.0,0.0,0.0,20221025,HOT_COLD_2_0,5ba1f6d0be5b826084cc7202
2,hot,Gopalpura_JAI,Monday,1400,Monday-1400,1100,57930.0,14,4137.857143,5.693801,34.877946,3.773585,0.0,0.0,0.0,0.0,0.0,20221025,HOT_COLD_2_0,5ba1f6d0be5b826084cc7202
3,hot,Bani Park_JAI,Monday,1400,Monday-1400,261,16151.0,4,4037.750000,5.556051,40.433997,5.660377,0.0,0.0,0.0,0.0,0.0,20221025,HOT_COLD_2_0,5ba1f6d0be5b826084cc7202
4,hot,Mansarovar 2_JAI,Monday,1400,Monday-1400,249,13842.0,4,3460.500000,4.761740,45.195736,7.547170,0.0,0.0,0.0,0.0,0.0,20221025,HOT_COLD_2_0,5ba1f6d0be5b826084cc7202


In [15]:
df_mismatch = df_mismatch[df_mismatch['run_date'] == df_mismatch['run_date'].max()]

In [16]:
#rename column
df_mismatch = df_mismatch.rename(columns = {'week_period':'weekday_name', 'pickupclusters':'pickup_cluster'})
df_mismatch['hour'] = df_mismatch['time_period'].apply(lambda x: x[:2])
df_mismatch.head()

,hotness_category,pickup_cluster,weekday_name,time_period,temporal_id,total_footfall,total_revenue,hex_count,total_revenue_normalized,normalized_revenue_contribution_percentage,cum_revenue_contribution_percentage,revenue_contribution_percentile_rank,temporal_stress,temporal_mismatch,spatial_stress,spatial_mismatch,cluster_hotness_score,run_date,strategy_version,service_detail_id,hour
7457,super-cold,Jaipur National University,Wednesday,0900,Wednesday-0900,0,0.0,0,0.0,0.0,0.0,0.0,0.056516,0.127886,0.097941,0.107986,0.097582,20230314,HOT_COLD_3_0,5ba1f6d0be5b826084cc7202,09
7806,cold,Jaipur National University,Wednesday,1300,Wednesday-1300,0,0.0,0,0.0,0.0,0.0,0.0,0.237141,0.312302,0.292818,0.279190,0.280362,20230314,HOT_COLD_3_0,5ba1f6d0be5b826084cc7202,13
7807,medium,Jaipur National University,Wednesday,1800,Wednesday-1800,0,0.0,0,0.0,0.0,0.0,0.0,0.254263,0.419976,0.263615,0.275454,0.303327,20230314,HOT_COLD_3_0,5ba1f6d0be5b826084cc7202,18
7808,cold,Jaipur National University,Wednesday,2100,Wednesday-2100,0,0.0,0,0.0,0.0,0.0,0.0,0.199311,0.522343,0.119705,0.138122,0.244870,20230314,HOT_COLD_3_0,5ba1f6d0be5b826084cc7202,21
7809,medium,Jaipur National University,Wednesday,2300,Wednesday-2300,0,0.0,0,0.0,0.0,0.0,0.0,0.253304,0.593625,0.255985,0.274401,0.344329,20230314,HOT_COLD_3_0,5ba1f6d0be5b826084cc7202,23


In [17]:
df_morning = df_morning.merge(df_mismatch[['weekday_name', 'hour', 'pickup_cluster', 'cluster_hotness_score']],
                               on = ['weekday_name', 'hour', 'pickup_cluster'], how = 'left')
df_evening = df_evening.merge(df_mismatch[['weekday_name', 'hour', 'pickup_cluster', 'cluster_hotness_score']],
                               on = ['weekday_name', 'hour', 'pickup_cluster'], how = 'left')
df_evening.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,second,minute,date,count,pickup_cluster,drop_cluster,num_of_orders,weekday_name,cluster_hotness_score
0,640754397998491eec413d4d,1.684,883da20b5dfffff,26.840919,75.805298,Jaipur,883da20b5dfffff,26.840910,883da20b5dfffff,26.840931,75.805541,883da2569dfffff,26.820506,75.761160,75.805420,204153,20,2030,Link,20:59,2,20230307,customerCancelled,Wrong pickup location,53,41,07,1,Siddharth Nagar_JAI,Sanganer_JAI,8,Tuesday,0.371344
1,64074eb296550e0e81d42a84,2.691,883da2198bfffff,26.870806,75.781776,Jaipur,883da2198bfffff,26.870818,883da2198bfffff,26.870437,75.781724,883da218a7fffff,26.888162,75.802744,75.781779,201818,20,2015,Link,20:00,2,20230307,customerCancelled,Taking longer than expected,18,18,07,1,Triveni_Nagar_JAI,,17,Tuesday,0.550284
2,64075499483a382df32b6e84,3.172,883da21ab5fffff,26.941658,75.776947,Jaipur,883da21ab5fffff,26.941648,883da21ab5fffff,26.941647,75.776970,883da21ac5fffff,26.962606,75.738029,75.776962,204329,20,2030,Link,20:59,2,20230307,customerCancelled,Taking longer than expected,29,43,07,1,Vidhyadhar Nagar_JAI,Nirmal_Vihar_JAI,5,Tuesday,0.398078
3,640754da687e977e559b9290,1.731,883da20b19fffff,26.834332,75.834610,Jaipur,883da20b59fffff,26.844872,883da20b59fffff,26.844886,75.814416,883da20b19fffff,26.834332,75.834610,75.814407,204434,20,2030,Link,20:59,2,20230307,dropped,None,34,44,07,1,Malviya_Nagar_JAI 2,Siddharth Nagar_JAI,7,Tuesday,0.598544
4,640756e396550e0e81d43808,NaN,883da20a61fffff,26.857084,75.821190,Jaipur,883da20a63fffff,26.860607,883da20a63fffff,26.859827,75.824470,883da218b5fffff,26.893931,75.817889,75.825165,205315,20,2045,Link,20:59,2,20230307,customerCancelled,order cancelled before rider accepted,15,53,07,1,Malviya_Nagar_JAI 2,Tilak nagar_JAI,3,Tuesday,0.598544


In [18]:
df_morning_filter = df_morning[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'weekday_name', 'cluster_hotness_score', 'order_status']]
df_evening_filter = df_evening[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'weekday_name', 'cluster_hotness_score', 'order_status']]

In [19]:
df_evening_filter.to_csv('jaipur_demand_supply_evening_vw2.csv', index = False)
df_morning_filter.to_csv('jaipur_demand_supply_morning_vw2.csv', index = False)

In [18]:
#Make a copy of the dataset and view it
df_online_captains = df_online_captains_copy.copy()
df_online_captains.head()

,yyyymmdd,hour,cluster,Online_Captains
0,20230301,17,Nungambakkam,237
1,20230228,17,Nungambakkam,231
2,20230301,21,Teynampet,175
3,20230228,20,T Nagar,322
4,20230301,18,Kotturpuram,66


In [19]:
df_online_captains.groupby(['yyyymmdd', 'hour'])['Online_Captains'].sum().reset_index()

,yyyymmdd,hour,Online_Captains
0,20230227,17,1338
1,20230227,18,1595
2,20230227,19,1795
3,20230227,20,1689
4,20230227,21,1399
5,20230228,17,1257
6,20230228,18,1519
7,20230228,19,1787
8,20230228,20,1624
9,20230228,21,1371


In [20]:
#Rename cluster column
df_online_captains = df_online_captains.rename(columns = {'cluster':'pickup_cluster'})
df_online_captains.head()

,yyyymmdd,hour,pickup_cluster,Online_Captains
0,20230301,17,Nungambakkam,237
1,20230228,17,Nungambakkam,231
2,20230301,21,Teynampet,175
3,20230228,20,T Nagar,322
4,20230301,18,Kotturpuram,66


In [21]:
#Merge online captains with demand data
df_morning_grouped = df_morning_grouped.merge(df_online_captains, on = ['yyyymmdd', 'hour', 'pickup_cluster'], how = 'left')
df_evening_grouped = df_evening_grouped.merge(df_online_captains, on = ['yyyymmdd', 'hour', 'pickup_cluster'], how = 'left')
df_morning_grouped = df_morning_grouped.rename(columns = {'order_id':'num_of_orders'})
df_evening_grouped = df_evening_grouped.rename(columns = {'order_id':'num_of_orders'})
df_morning_grouped['demand_to_supply_ratio'] = round(df_morning_grouped['num_of_orders']/df_morning_grouped['Online_Captains'], 4)
df_evening_grouped['demand_to_supply_ratio'] = round(df_evening_grouped['num_of_orders']/df_evening_grouped['Online_Captains'], 4)

df_morning_grouped = df_morning_grouped.sort_values(by = 'demand_to_supply_ratio', ascending = False)
df_evening_grouped = df_evening_grouped.sort_values(by = 'demand_to_supply_ratio', ascending = False)
df_evening_grouped.head()

,yyyymmdd,hour,pickup_cluster,num_of_orders,Online_Captains,demand_to_supply_ratio
169,20230302,18,Thousand Lights Metro,92,43,2.1395
119,20230301,18,Thousand Lights Metro,87,50,1.7400
164,20230302,18,Nungambakkam,392,235,1.6681
100,20230301,17,Alwarpet,71,44,1.6136
110,20230301,18,Alwarpet,118,76,1.5526


In [27]:
df_evening_grouped.groupby(['hour'])['demand_to_supply_ratio'].mean()

hour
17    0.918695
18    1.022725
19    0.714440
20    0.515355
21    0.482165
Name: demand_to_supply_ratio, dtype: float64

In [22]:
df_morning = df_morning.merge(df_morning_grouped, on = ['yyyymmdd', 'pickup_cluster', 'hour'], how = 'left')
df_evening = df_evening.merge(df_evening_grouped, on = ['yyyymmdd', 'pickup_cluster', 'hour'], how = 'left')
df_morning.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,quarter_hour,service_obj_service_name,time_bucket,weekday,order_status,cancel_reason,second,minute,date,count,drop_cluster,num_of_orders,yyyymmdd,hour,pickup_cluster,Online_Captains,demand_to_supply_ratio


In [23]:
#Create a column for weekdays
df_morning['weekday_name'] = df_morning['weekday'].apply(lambda x: calendar.day_name[(x-1)])
df_evening['weekday_name'] = df_evening['weekday'].apply(lambda x: calendar.day_name[(x-1)])
df_morning.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,pickup_location_hex_8,pickup_location_latitude,pickup_location_longitude,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_longitude,hhmmss,quarter_hour,service_obj_service_name,time_bucket,weekday,order_status,cancel_reason,second,minute,date,count,drop_cluster,num_of_orders,yyyymmdd,hour,pickup_cluster,Online_Captains,demand_to_supply_ratio,weekday_name


In [24]:
num_of_hex_morning = df_morning.groupby(['pickup_cluster'])['pickup_location_hex_8'].nunique().reset_index()
num_of_hex_evening = df_evening.groupby(['pickup_cluster'])['pickup_location_hex_8'].nunique().reset_index()
num_of_hex_morning = num_of_hex_morning.rename(columns = {'pickup_location_hex_8':'num_of_hexes'})
num_of_hex_evening = num_of_hex_evening.rename(columns = {'pickup_location_hex_8':'num_of_hexes'})

df_morning = df_morning.merge(num_of_hex_evening, on = ['pickup_cluster'], how = 'left')
df_evening = df_evening.merge(num_of_hex_evening, on = ['pickup_cluster'], how = 'left')
df_morning['online_captains_per_hex'] = round(df_morning['Online_Captains']/df_morning['num_of_hexes'], 4)
df_evening['online_captains_per_hex'] = round(df_evening['Online_Captains']/df_evening['num_of_hexes'], 4)

In [25]:
df_morning_filter = df_morning[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'Online_Captains', 'demand_to_supply_ratio', 'weekday_name', 'online_captains_per_hex']]
df_evening_filter = df_evening[['captain_location_hex_8', 'captain_location_latitude', 'captain_location_longitude', 'pickup_location_hex_8',
 'pickup_location_latitude', 'pickup_location_longitude', 'pickup_cluster', 'hour', 'yyyymmdd', 'drop_location_hex_8', 'drop_location_latitude',
  'drop_location_longitude', 'num_of_orders', 'drop_cluster', 'Online_Captains', 'demand_to_supply_ratio', 'weekday_name', 'online_captains_per_hex']]

In [26]:
df_evening_filter.to_csv('chennai_demand_supply_evening.csv', index = False)